<a href="https://colab.research.google.com/github/franckglastre/NLP-Datascientest/blob/Nicolas/Py_Easymail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Structuration DataSet (A ne plus relancer)

In [ ]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# On concaténe les 6 fichiers
import pandas as pd
df_0 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_0.csv")
df_1 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_1.csv")
df_2 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_2.csv")
df_3 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_3.csv")
df_4 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_4.csv")
df_5 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_5.csv")
# df = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron.csv", nrows=1500)
frames = [df_0,df_1,df_2,df_3,df_4,df_5]
df = pd.concat(frames)
# la selection ci-dessous a permis de détecter 3 doublons (toutes colonnes identiques)
# df = df.loc[(df['subject'] == "Sagewood Town Homes")]
print(df.shape)
# df = df_emails.loc[(df['subject'] == "Sagewood Town Homes")]
df.drop('Unnamed: 0', axis=1, inplace=True)
# suppression des doublons (sur toutes les colonnes sauf index)
df.drop_duplicates(subset=['date','sender', 'recipient1', 'subject','text'],keep = 'first', inplace=True)
# on renomme les colonnes
df = df.rename(columns={'sender': 'from', 'recipient1': 'to','subject': 'header', 'text': 'body'})
# et export en csv du résultat.
file_name = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique.csv"
df.to_csv(file_name, encoding='utf-8', index=False)
print(df.shape)
# Taille du fichier divisée par 2 environ

(405968, 6)
(227032, 5)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227032 entries, 0 to 11736
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    227032 non-null  object
 1   from    227032 non-null  object
 2   to      227032 non-null  object
 3   header  218946 non-null  object
 4   body    227032 non-null  object
dtypes: object(5)
memory usage: 10.4+ MB


# Import - Data Preprocessing structures

Import Spacy

In [ ]:
#!pip install -U spacy
#!python -m spacy download en_core_web_sm


In [2]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
stopwords=list(STOP_WORDS)
from string import punctuation
punctuation=punctuation+ '\n'

Mount Gdrive files

In [3]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd
# Pour limiter les temps de calcul on travaille uniquement sur les 'nrows' premiers mails
df = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique.csv", nrows=100)
df.head()

Mounted at /content/drive


,date,from,to,header,body
0,2001-05-14 16:39:00-07:00,phillip.allen@enron.com,tim.belden@enron.com,NaN,"['', 'Here is our forecast', '', ' ']"
1,2001-05-04 13:51:00-07:00,phillip.allen@enron.com,john.lavorato@enron.com,Re:,"['', 'Traveling to have a business meeting tak..."
2,2000-10-18 03:00:00-07:00,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,"['', 'test successful. way to go!!!']"
3,2000-10-23 06:13:00-07:00,phillip.allen@enron.com,randall.gay@enron.com,NaN,"['', 'Randy,', '', ' Can you send me a schedul..."
4,2000-08-31 05:07:00-07:00,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,"['', ""Let's shoot for Tuesday at 11:45. ""]"


Overview dataset

In [4]:
# nbre d'expéditeurs distincts
len(df['from'].unique())

1

In [5]:
# nbre de destinataires distincts
len(df['to'].unique())

49

Rajout colonnes pour traitements ultérieurs

In [6]:
import numpy as np
df = df.replace('NaN', np.NaN)
# df_emails = df[df['header'].notna()]
df_emails = df
df_emails['NER_header'] = ''
df_emails['NER_body'] = ''
df_emails['body_dict'] = ''
df_emails['summary_TFIDF'] = ''
df_emails['summary_spacy'] = ''
df_emails['summary_abstractive'] = ''
df_emails['summary_gensim'] = ''
df_emails.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   date                 100 non-null    object
 1   from                 100 non-null    object
 2   to                   100 non-null    object
 3   header               60 non-null     object
 4   body                 100 non-null    object
 5   NER_header           100 non-null    object
 6   NER_body             100 non-null    object
 7   body_dict            100 non-null    object
 8   summary_TFIDF        100 non-null    object
 9   summary_spacy        100 non-null    object
 10  summary_abstractive  100 non-null    object
 11  summary_gensim       100 non-null    object
dtypes: object(12)
memory usage: 9.5+ KB


In [7]:
df_emails.columns

Index(['date', 'from', 'to', 'header', 'body', 'NER_header', 'NER_body',
       'body_dict', 'summary_TFIDF', 'summary_spacy', 'summary_abstractive',
       'summary_gensim'],
      dtype='object')

In [8]:
#
# A ce stade le fichier a la structure nécessaire à Melusine pour le nettoyage
#

# NER with Spacy

In [9]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
spacy_stopwords = nlp.Defaults.stop_words
stopwords = spacy_stopwords
print(spacy_stopwords)


deb = 0
fin = df_emails.shape[0]
for x in range(deb,fin):
# subject
  subj = df_emails.iloc[x]['header']
#  print(x,' - ',subj)
  doc_subj = nlp(str(subj))
  df_emails['NER_header'][x] = doc_subj.ents

# body
  subj = df_emails.iloc[x]['body']
#  print(x,' - ',subj)
  doc_subj = nlp(str(subj))
  df_emails['NER_body'][x] = doc_subj.ents

{'when', 'not', 'who', 'hundred', 'was', 'latterly', 'perhaps', 'are', 'two', 'say', 'my', 'along', '‘s', '‘ve', 'behind', 'well', 'among', 'thence', 'whom', 'beforehand', 'beyond', "'ll", 'ca', 'until', '’re', 'eleven', 'both', 'them', 'while', 'might', 'sometime', 'whereafter', 'how', 'either', 'ourselves', 'wherever', 'ours', 'various', "'s", 'amount', 'name', 'beside', 'n‘t', 'between', 'can', 'per', 'go', 'around', 'using', 'those', 'some', 'quite', 'whereas', "'m", 'hereupon', 'often', 'once', 'put', 'move', 'against', 'what', 'being', '’m', 'therein', 'thru', 'whenever', 'so', 'doing', 'and', 'though', "'ve", 'really', 'is', 'be', 'elsewhere', 'besides', 'everywhere', 'last', 'there', 'such', 'whereby', 'give', 'becoming', 'else', '‘d', 'since', 'to', 'always', 'itself', 'own', 'however', 'get', 'us', 'via', 'yours', 'front', 'someone', 'on', 'you', 'out', 'neither', 'therefore', 'after', 'something', 'nevertheless', 'which', 'under', 'themselves', 'whatever', 'than', 'during', 

In [10]:
df_emails[['header','NER_header','NER_body']][0:40]

,header,NER_header,NER_body
0,NaN,(),()
1,Re:,(),"((Austin),)"
2,Re: test,(),()
3,NaN,(),"((Randy), (Patti, S))"
4,Re: Hello,(),"((Tuesday), (11:45))"
5,Re: Hello,(),"((next, Tuesday), (Thursday))"
6,NaN,(),"((Phillip, Allen), (Mike, Grigsby), (Keith, Ho..."
7,Re: PRC review - phone calls,"((PRC),)","((10, and, 11:30, '),)"
8,Re: High Speed Internet Access,(),"((ISP), (2), (IP), (DNS), (3))"
9,FW: fixed forward or other Collar floor gas pr...,"((FW, :), (Collar))","((Phillip, K, Allen, /, HOU, /, ECT), (01:42, ..."


In [11]:
id = 12
doc= nlp(df_emails['header'][id])
displacy.render(doc, style = "ent",jupyter = True)
print("----- eMail ",id," -----")
doc= nlp(df_emails['body'][id])
displacy.render(doc, style = "ent",jupyter = True)
print("\n")

----- eMail  12  -----


In [12]:
# A ce stade le fichier a la structure attendue par melusine + l'identification des NER par Spacy
df_emails[['header','NER_header','body','NER_body']].iloc[id]

header              Consolidated positions: Issues & To Do list
NER_header                               ((Issues, &, To, Do),)
body          ['', '---------------------- Forwarded by Phil...
NER_body      ((Phillip, K, Allen, /, HOU, /, ECT), (02:16, ...
Name: 12, dtype: object

In [13]:
df_emails[['header','body','body_dict','summary_TFIDF','NER_body']][0:30]

,header,body,body_dict,summary_TFIDF,NER_body
0,NaN,"['', 'Here is our forecast', '', ' ']",,,()
1,Re:,"['', 'Traveling to have a business meeting tak...",,,"((Austin),)"
2,Re: test,"['', 'test successful. way to go!!!']",,,()
3,NaN,"['', 'Randy,', '', ' Can you send me a schedul...",,,"((Randy), (Patti, S))"
4,Re: Hello,"['', ""Let's shoot for Tuesday at 11:45. ""]",,,"((Tuesday), (11:45))"
5,Re: Hello,"['', 'Greg,', '', ' How about either next Tues...",,,"((next, Tuesday), (Thursday))"
6,NaN,"['', 'Please cc the following distribution lis...",,,"((Phillip, Allen), (Mike, Grigsby), (Keith, Ho..."
7,Re: PRC review - phone calls,"['', 'any morning between 10 and 11:30']",,,"((10, and, 11:30, '),)"
8,Re: High Speed Internet Access,"['', '1. login: pallen pw: ke9davis', '', "" I...",,,"((ISP), (2), (IP), (DNS), (3))"
9,FW: fixed forward or other Collar floor gas pr...,"['', '---------------------- Forwarded by Phil...",,,"((Phillip, K, Allen, /, HOU, /, ECT), (01:42, ..."


In [14]:
text = str(df_emails['summary_TFIDF'][32])
text="""The human coronavirus was first diagnosed in 1965 by Tyrrell and Bynoe from the respiratory tract sample of an adult with a common cold cultured on human embryonic trachea.1 Naming the virus is based on its crown-like appearance on its surface.2 Coronaviruses (CoVs) are a large family of viruses belonging to the Nidovirales order, which includes Coronaviridae, Arteriviridae, and Roniviridae families.3 Coronavirus contains an RNA genome and belongs to the Coronaviridae family.4 This virus is further subdivided into four groups, ie, the α, β, γ, and δ coronaviruses.5 α- and β-coronavirus can infect mammals, while γ- and δ- coronavirus tend to infect birds.6 Coronavirus in humans causes a range of disorders, from mild respiratory tract infections, such as the common cold to lethal infections, such as the severe acute respiratory syndrome (SARS), Middle East respiratory syndrome (MERS) and Coronavirus disease 2019 (COVID-19). The coronavirus first appeared in the form of severe acute respiratory syndrome coronavirus (SARS-CoV) in Guangdong province, China, in 20027 followed by Middle East respiratory syndrome coronavirus (MERS-CoV) isolated from the sputum of a 60-year-old man who presented symptoms of acute pneumonia and subsequent renal failure in Saudi Arabia in 2012.8 In December 2019, a β-coronavirus was discovered in Wuhan, China. The World Health Organization (WHO) has named the new disease as Coronavirus disease 2019 (COVID-19), and Coronavirus Study Group (CSG) of the International Committee has named it as SARS-CoV-2.9,10 Based on the results of sequencing and evolutionary analysis of the viral genome, bats appear to be responsible for transmitting the virus to humans"""
print(text)

The human coronavirus was first diagnosed in 1965 by Tyrrell and Bynoe from the respiratory tract sample of an adult with a common cold cultured on human embryonic trachea.1 Naming the virus is based on its crown-like appearance on its surface.2 Coronaviruses (CoVs) are a large family of viruses belonging to the Nidovirales order, which includes Coronaviridae, Arteriviridae, and Roniviridae families.3 Coronavirus contains an RNA genome and belongs to the Coronaviridae family.4 This virus is further subdivided into four groups, ie, the α, β, γ, and δ coronaviruses.5 α- and β-coronavirus can infect mammals, while γ- and δ- coronavirus tend to infect birds.6 Coronavirus in humans causes a range of disorders, from mild respiratory tract infections, such as the common cold to lethal infections, such as the severe acute respiratory syndrome (SARS), Middle East respiratory syndrome (MERS) and Coronavirus disease 2019 (COVID-19). The coronavirus first appeared in the form of severe acute respi

In [15]:
# import spacy
# from spacy import displacy
# nlp = spacy.load('en_core_web_sm')
# spacy_stopwords = nlp.Defaults.stop_words
# stopwords = spacy_stopwords
# print(spacy_stopwords)

# Data cleaning

In [16]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import re

def data_clean (tokens):
# Suppression des caractères spéciaux ou inutiles \"',]
#  print("Data cleaning --\n")
#  print("Avant :\n",type(tokens)," ",tokens)
  # r = re.compile(r"[\\\''\",]")
  r = re.compile(r"[\\\''\",]")
  texte=r.sub('', tokens)
  texte = sent_tokenize(texte)
#  print("Après :\n",type(texte)," ",texte)
  return texte

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
def remove_stop_words(text,stopwords):
  lst=[]
  for token in str(text).split():
    if token.lower() not in stopwords:    #checking whether the word is not 
      lst.append(token)                    #present in the stopword list.      
    else:
      print("suppression du stopword : ",token)
  # Join items in the list
#  print("Original text before removing stopwords  :   \n",text)
  result = ' '.join(lst)
#  print("Text after removing stopwords  :   \n",result)
  return result

In [18]:
# Transforme Text en Dictionnaire - Key = indice de phrase - Value = Mots de la phrase
import re
from nltk.tokenize import sent_tokenize,word_tokenize

def dictionarize(article):
    dico={}
    phrases=sent_tokenize(article)
    phr2=[]
    for i, phr in enumerate(phrases):
        import re
        phr=re.sub(pattern ="[^a-zA-Z]", repl = ' ', string = phr)
        phr2.append(phr)
    
        phr=word_tokenize(phr)
        phr_lower=[w.lower() for w in phr]
        if len(phr_lower)>4:      
            dico[i]=phr_lower    
    return dico



In [19]:
df_emails.shape[0]

100

# Summary (Extractive methodS)

In [20]:
import pandas as pd
import numpy as np


def TF(token, artikle):
    """
    Calcule le score TF d'un mot dans un artikle
    
    token : Mot dont le score TF doit être calculé.
    
    artikle : Dictionnaire généré à partir d'un texte.
    """
    f = 0
    for key in artikle:
        for work in artikle[key]:
            if work == token:
                f += 1 
    return np.log(f+1)

def IDF(token, corpus):
    """
    Calcule le score IDF d'un mot dans un corpus d'artikles.
    
    token : Mot dont le score IDF doit être calculé.
    
    corpus : Liste d'artikles.
    """
    N = len(corpus)
    d=0
    present = False
    
    for artikle in corpus:
        for key in artikle:
            if token in artikle[key]:
                present = True
        d += int(present)
        present = False
                
    return np.log(N/(d+1) +1)

def TFIDF(token, artikle, corpus):
    """
    Calcule le score TF-IDF d'un mot dans un texte.
    
    token : mot dont le score doit être calculé.
    
    artikle : artikle qui servira à calculer le score du mot dans cet artikle.
    
    corpus : Liste d'artikles formant le corpus.
    """
    return TF(token, artikle)*IDF(token, corpus)

def score_sentence(corpus, artikle, sentence):
    """
    Calcule le score d'une phrase.
    
    corpus : Liste d'artikles.
    
    artikle : Dictionnaire de phrases.
    
    sentence : Phrase sous forme de liste de mots.
    """
    score_sentence = []
    for word in sentence :
        score_word = TFIDF(word, artikle, corpus)
        score_sentence.append(score_word)
    return np.mean(score_sentence)

def best_sentences(scores_artikle, nb_sentences):
    """
    Retourne les indices des phrases les plus importantes en fonction des scores obtenus.
    
    scores_artikle : Liste des scores de chaque phrase dans un texte.
    
    nb_sentences : Nombre de phrases à sélectionner.
    """
    
    return sorted(np.argsort(scores_artikle)[-nb_sentences:])

def summarize_TFIDF(i, n_sentences, df):
    """
    Synthèse extractive d'un article par la méthode TF-IDF.
    
    i : indice de l'article dans le corpus.
    
    n_sentences : nombre de phrases à conserver.
    
    df : DataFrame contenant les artikles dans une colonne 'Artikle'.    
    """
    corpus = df['body_dict']
    artikle = corpus[i]
    texte=df['body'][i]
    
    
    # Calcul du score de chaque phrase de l'artikle
    scores_artikle = [score_sentence(corpus, artikle, sentence) for sentence in artikle.values()]
 
    
    # Extraction des indices des phrases ayant les meilleurs scores
    result = best_sentences(scores_artikle, n_sentences)
#    print('best sentences :',result)
    
    # Séparation de phrases l'article original 
    tokenized_article = sent_tokenize(texte)
    
    # Extraction des phrases les plus importantes
    summary = [tokenized_article[i] for i in result]
#    print("[",i,"] :",summary)
    return summary

def summarize_spacy (text):
  # https://www.numpyninja.com/post/text-summarization-through-use-of-spacy-library
  nlp = spacy.load('en_core_web_sm')
  doc= nlp(text)
  tokens=[token.text for token in doc]
  print("Tokens : \n",tokens)

  # calcul frequence de mots
  word_frequencies={}
  for word in doc:
    if word.text.lower() not in stopwords:
      if word.text.lower() not in punctuation:
        if word.text not in word_frequencies.keys():
          word_frequencies[word.text] = 1
        else:
          word_frequencies[word.text] += 1
#  print("\n word_frequencies : ",word_frequencies)

  # normalisation des frequence de mots
  max_frequency=max(word_frequencies.values())
  for word in word_frequencies.keys():
    word_frequencies[word]=word_frequencies[word]/max_frequency
#  print("\n Normalized word_frequencies : \n",word_frequencies)
  # Sentences token
  sentence_tokens= [sent for sent in doc.sents]
  print(sentence_tokens)
  # Calculate the most important sentences by adding the word frequencies in each sentence.
  sentence_scores = {}
  for sent in sentence_tokens:
    for word in sent:
      if word.text.lower() in word_frequencies.keys():
        if sent not in sentence_scores.keys():                            
          sentence_scores[sent]=word_frequencies[word.text.lower()]
        else:
          sentence_scores[sent]+=word_frequencies[word.text.lower()]

  # identifier 30% du texte avec score maximum
  from heapq import nlargest
  select_length=int(len(sentence_tokens)*0.25)
  select_length
  summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
  final_summary=[word.text for word in summary]
  summary=''.join(final_summary)
  return summary



Test unitaire

In [21]:
id = 32
text = df_emails['body'].iloc[id]
text = data_clean(text)
# text = remove_stop_words(text,stopwords)

# Dictionnarize the text to be used with TF-IDF
df_emails['body_dict'][id] = dictionarize(str(text))
# for cle, valeur in text.items():
#   print("clé", cle, "vaut", valeur)
#  Extractive summary using TF-IDF
df_emails['summary_TFIDF'][id] = summarize_TFIDF(id, 3, df_emails)

# Extractive summary using Spacy
df_emails['summary_spacy'][id] = summarize_spacy (str(text))



Tokens : 
 ['[', "'", '[', '----------------------', 'Forwarded', 'by', 'Phillip', 'K', 'Allen', '/', 'HOU', '/', 'ECT', 'on', '09/26/2000', ' ', '11:57', 'AM', '---------------------------', '  ', 'BS', 'Stone', '<', 'bs_stone@yahoo.com', '>', 'on', '09/26/2000', '04:47:40', 'AM', 'To', ':', 'jeff', '<', 'jeff@freeyellow.com', '>', 'cc', ':', 'Phillip', 'K', 'Allen', '<', 'Phillip.K.Allen@enron.com', '>', ' ', 'Subject', ':', 'closing', '   ', 'Jeff', ' ', '?', "'", ',', "'", 'Is', 'the', 'closing', 'today', '?', '?', "'", ',', "'", 'After', 'reviewing', 'the', ' ', 'agreement?I', 'find', 'it', 'is', 'nt', 'binding', ' ', 'as', 'far', 'as', 'I', 'can', 'determine', '.', '?', "'", ',', "'", 'It', 'is', ' ', 'too', 'vague', 'and', 'it', 'does', 'nt', 'sound', 'like', ' ', 'anything', 'an', 'attorney', 'or', 'title', 'company', ' ', 'would?draft', 'for', 'a', 'real', 'estate', ' ', 'closing', '--', 'but', 'of', 'course', 'I', 'could', 'be', ' ', 'wrong', '.', '?', ' ', '?', "'", ',', "'"

In [22]:
len(df_emails['body'][id]),df_emails['body'][id]

(1966,
 '[\'\', \'---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/26/2000 \', \'11:57 AM ---------------------------\', \'\', \'\', \'"BS Stone" <bs_stone@yahoo.com> on 09/26/2000 04:47:40 AM\', \'To: "jeff" <jeff@freeyellow.com>\', \'cc: "Phillip K Allen" <Phillip.K.Allen@enron.com> \', \'Subject: closing\', \'\', \'\', \'\', \'Jeff, \', \'?\', "Is the closing today?? After reviewing the  agreement?I find it isn\'t binding ", "as far as I can determine.? It is  too vague and it doesn\'t sound like ", \'anything an attorney or title company  would?draft for a real estate \', \'closing--but, of course, I could be  wrong.? \', \'?\', \'If this?closing is going to take place without  this agreement then there is \', "no point in me  following up on this?document\'s validity.? ", \'?\', "I will just need to go back to my closing documents  and see what\'s there and ", \'find out where I am with that and deal with this as  best I can.\', \'?\', \'I guess I was expecting so

In [23]:
len(df_emails['summary_TFIDF'][id])/len(df_emails['body'][id]),df_emails['summary_TFIDF'][id]

(0.001525940996948118,
 ["', '?",
  '\', "I will just need to go back to my closing documents  and see what\'s there and ", \'find out where I am with that and deal with this as  best I can.',
  "', 'I guess I was expecting something that would be an  exhibit to a recordable ', 'document or something a little more exact, or  rather?sort of a contract.?"])

In [24]:
len(df_emails['summary_spacy'][id])/len(df_emails['body'][id]),df_emails['summary_spacy'][id]

(0.26805696846388605,
 " ?', 'I will just need to go back to my closing documents  and see whats there and  find out where I am with that and deal with this as  best I can. ?', 'I guess I was expecting something that would be an  exhibit to a recordable  document or something a little more exact or  rather?sort of a contract.?', 'This isnt either.?', 'I tried to get a  real estate atty on the phone last  night but he was out of pocket.?', 'I  talked to a crim.', 'atty friend and he said  this is out of his area but doesnt  sound binding to him.?")

# Boucle de traitement

In [25]:
# Nettoyage : Détokenisation et retokenisation en sentences
deb = 0
fin = df_emails.shape[0]
fin = 50
for id in range(deb,fin):
  print("\n id : ",id)
  text = df_emails['body'].iloc[id]
  print("Data cleaning")
  text = data_clean(text)
# text = remove_stop_words(text,stopwords)
  print("TF_IDF")
# Dictionnarize the text to be used with TF-IDF
  df_emails['body_dict'][id] = dictionarize(str(text))
# for cle, valeur in text.items():
#   print("clé", cle, "vaut", valeur)
#  Extractive summary using TF-IDF
  df_emails['summary_TFIDF'][id] = summarize_TFIDF(id, 3, df_emails)
  print("Extractive sum with Spacy")
# Extractive summary using Spacy
  df_emails['summary_spacy'][id] = summarize_spacy (str(text))



 id :  0
Data cleaning
TF_IDF
Extractive sum with Spacy
Tokens : 
 ['[', "'", '[', 'Here', 'is', 'our', 'forecast', '  ', ']', "'", ']']
[['[ Here is our forecast   ]']]

 id :  1
Data cleaning
TF_IDF
Extractive sum with Spacy
Tokens : 
 ['[', "'", '[', 'Traveling', 'to', 'have', 'a', 'business', 'meeting', 'takes', 'the', 'fun', 'out', 'of', 'the', 'trip', '.', "'", ',', "'", 'Especially', 'if', 'you', 'have', 'to', 'prepare', 'a', 'presentation', '.', "'", ',', "'", 'I', 'would', 'suggest', 'holding', 'the', 'business', 'plan', 'meetings', 'here', 'then', 'take', 'a', 'trip', 'without', 'any', 'formal', 'business', 'meetings', '.', "'", ',', "'", 'I', 'would', 'even', 'try', 'and', 'get', 'some', 'honest', 'opinions', 'on', 'whether', 'a', 'trip', 'is', 'even', 'desired', 'or', 'necessary', '.', "'", ',', "'", 'As', 'far', 'as', 'the', 'business', 'meetings', 'I', 'think', 'it', 'would', 'be', 'more', 'productive', 'to', 'try', 'and', 'stimulate', 'discussions', 'across', 'the', 'di

In [26]:
text

['[ http://www.hearme.com/vc2/?chnlOwnr=pallen@enron.com]']

In [27]:
df_emails.shape

(100, 12)

In [28]:
df_emails['body'][32]

'[\'\', \'---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/26/2000 \', \'11:57 AM ---------------------------\', \'\', \'\', \'"BS Stone" <bs_stone@yahoo.com> on 09/26/2000 04:47:40 AM\', \'To: "jeff" <jeff@freeyellow.com>\', \'cc: "Phillip K Allen" <Phillip.K.Allen@enron.com> \', \'Subject: closing\', \'\', \'\', \'\', \'Jeff, \', \'?\', "Is the closing today?? After reviewing the  agreement?I find it isn\'t binding ", "as far as I can determine.? It is  too vague and it doesn\'t sound like ", \'anything an attorney or title company  would?draft for a real estate \', \'closing--but, of course, I could be  wrong.? \', \'?\', \'If this?closing is going to take place without  this agreement then there is \', "no point in me  following up on this?document\'s validity.? ", \'?\', "I will just need to go back to my closing documents  and see what\'s there and ", \'find out where I am with that and deal with this as  best I can.\', \'?\', \'I guess I was expecting something 

In [29]:
id = 32
text = df_emails['body'][32]
df_emails['summary_spacy'][id] = summarize_spacy (text)

Tokens : 
 ['[', "''", ',', "'", '----------------------', 'Forwarded', 'by', 'Phillip', 'K', 'Allen', '/', 'HOU', '/', 'ECT', 'on', '09/26/2000', "'", ',', "'", '11:57', 'AM', '---------------------------', "'", ',', "''", ',', "''", ',', "'", '"', 'BS', 'Stone', '"', '<', 'bs_stone@yahoo.com', '>', 'on', '09/26/2000', '04:47:40', 'AM', "'", ',', "'", 'To', ':', '"', 'jeff', '"', '<', 'jeff@freeyellow.com', '>', "'", ',', "'", 'cc', ':', '"', 'Phillip', 'K', 'Allen', '"', '<', 'Phillip.K.Allen@enron.com', '>', "'", ',', "'", 'Subject', ':', 'closing', "'", ',', "''", ',', "''", ',', "''", ',', "'", 'Jeff', ',', "'", ',', "'", '?', "'", ',', '"', 'Is', 'the', 'closing', 'today', '?', '?', 'After', 'reviewing', 'the', ' ', 'agreement?I', 'find', 'it', 'is', "n't", 'binding', '"', ',', '"', 'as', 'far', 'as', 'I', 'can', 'determine', '.', '?', 'It', 'is', ' ', 'too', 'vague', 'and', 'it', 'does', "n't", 'sound', 'like', '"', ',', "'", 'anything', 'an', 'attorney', 'or', 'title', 'company

In [30]:
df_emails['summary_spacy'][id]

' I am in after 12 noon if there is any \', \'need to contact me regarding the  closing.\', \'?\', \'I really do not want to hold up anything or  generate more work for myself \', "and I don\'t want to insult or annoy anyone but  this paper really doesn\'t ", \'seem to be something required for a closing.?", \'?\', "I will just need to go back to my closing documents  and see what\'s there and ", \'find out where I am with that and deal with this as  best I can.\', \'?\', \'I guess I was expecting something that would be an  exhibit to a recordable \', \'document or something a little more exact, or  rather?sort of a contract.?It is  too vague and it doesn\'t sound like ", \'anything an attorney or title company  would?draft for a real estate \', \'closing--but, of course, I could be  wrong.?'

In [31]:
text

'[\'\', \'---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/26/2000 \', \'11:57 AM ---------------------------\', \'\', \'\', \'"BS Stone" <bs_stone@yahoo.com> on 09/26/2000 04:47:40 AM\', \'To: "jeff" <jeff@freeyellow.com>\', \'cc: "Phillip K Allen" <Phillip.K.Allen@enron.com> \', \'Subject: closing\', \'\', \'\', \'\', \'Jeff, \', \'?\', "Is the closing today?? After reviewing the  agreement?I find it isn\'t binding ", "as far as I can determine.? It is  too vague and it doesn\'t sound like ", \'anything an attorney or title company  would?draft for a real estate \', \'closing--but, of course, I could be  wrong.? \', \'?\', \'If this?closing is going to take place without  this agreement then there is \', "no point in me  following up on this?document\'s validity.? ", \'?\', "I will just need to go back to my closing documents  and see what\'s there and ", \'find out where I am with that and deal with this as  best I can.\', \'?\', \'I guess I was expecting something 

In [32]:
# texte fonction de résumé sur la ligne 'id'
#id = 12
#print(df_emails['body'][id])
#for cle, valeur in df_emails['body_dict'][id].items():
#        print("Ligne ", cle, "vaut", valeur)
#
#
#print('summary :\n\n',summarize(id, 3, df_emails))
#
# df_emails['summary'].iloc[id] = summarize(id, 5, df_emails)
# print(df_emails['summary'][id])

# SPACY - Summary (Abstractive / Word Embedding)

In [33]:
!pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 253 kB/s 
     |████████████████████████████████| 1.1 MB 13.7 MB/s 
     |████████████████████████████████| 4.2 MB 44.5 MB/s 
     |████████████████████████████████| 6.6 MB 39.1 MB/s 
     |████████████████████████████████| 596 kB 56.9 MB/s 
     |████████████████████████████████| 101 kB 10.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [34]:
# https://spacy.io/usage/embeddings-transformers
# https://spacy.io/api/transformer

# Summary (Abstractive method)

In [35]:
def summarize_gensim (text, ratio,word_count):
  # Summarization using 'gensim'
  import gensim
  print("text in def :",text)
  from gensim.summarization import summarize
  summary=summarize(text, ratio=0.3, word_count=100)
  return (summary)

def summarize_abstractive (text):
  from transformers import pipeline
#  print("Avant : ",text)
  # use the BART model, which is trained on the CNN/Daily Mail News Dataset, you can directly use the default parameters
  summarizer = pipeline("summarization")
  summarized = summarizer(text, min_length=75, max_length=300)
  summary=' '.join([str(i) for i in summarized])
#  print("Après : ",summary)
  return summary


In [36]:
# https://manusidtech.com/index.php/2021/08/14/complete-nlp-web-app-using-spacy-and-hugging-face-transformers/
#!pip install datasets transformers[sentencepiece]
text = str(df_emails['body'][32])
text="""The human coronavirus was first diagnosed in 1965 by Tyrrell and Bynoe from the respiratory tract sample of an adult with a common cold cultured on human embryonic trachea.1 Naming the virus is based on its crown-like appearance on its surface.2 Coronaviruses (CoVs) are a large family of viruses belonging to the Nidovirales order, which includes Coronaviridae, Arteriviridae, and Roniviridae families.3 Coronavirus contains an RNA genome and belongs to the Coronaviridae family.4 This virus is further subdivided into four groups, ie, the α, β, γ, and δ coronaviruses.5 α- and β-coronavirus can infect mammals, while γ- and δ- coronavirus tend to infect birds.6 Coronavirus in humans causes a range of disorders, from mild respiratory tract infections, such as the common cold to lethal infections, such as the severe acute respiratory syndrome (SARS), Middle East respiratory syndrome (MERS) and Coronavirus disease 2019 (COVID-19). The coronavirus first appeared in the form of severe acute respiratory syndrome coronavirus (SARS-CoV) in Guangdong province, China, in 20027 followed by Middle East respiratory syndrome coronavirus (MERS-CoV) isolated from the sputum of a 60-year-old man who presented symptoms of acute pneumonia and subsequent renal failure in Saudi Arabia in 2012.8 In December 2019, a β-coronavirus was discovered in Wuhan, China. The World Health Organization (WHO) has named the new disease as Coronavirus disease 2019 (COVID-19), and Coronavirus Study Group (CSG) of the International Committee has named it as SARS-CoV-2.9,10 Based on the results of sequencing and evolutionary analysis of the viral genome, bats appear to be responsible for transmitting the virus to humans"""
print(text)

print(summarize_abstractive (text))

The human coronavirus was first diagnosed in 1965 by Tyrrell and Bynoe from the respiratory tract sample of an adult with a common cold cultured on human embryonic trachea.1 Naming the virus is based on its crown-like appearance on its surface.2 Coronaviruses (CoVs) are a large family of viruses belonging to the Nidovirales order, which includes Coronaviridae, Arteriviridae, and Roniviridae families.3 Coronavirus contains an RNA genome and belongs to the Coronaviridae family.4 This virus is further subdivided into four groups, ie, the α, β, γ, and δ coronaviruses.5 α- and β-coronavirus can infect mammals, while γ- and δ- coronavirus tend to infect birds.6 Coronavirus in humans causes a range of disorders, from mild respiratory tract infections, such as the common cold to lethal infections, such as the severe acute respiratory syndrome (SARS), Middle East respiratory syndrome (MERS) and Coronavirus disease 2019 (COVID-19). The coronavirus first appeared in the form of severe acute respi

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

{'summary_text': ' The human coronavirus was first diagnosed in 1965 by Tyrrell and Bynoe from the respiratory tract sample of an adult with a common cold cultured on human embryonic trachea . Naming the virus is based on its crown-like appearance on its surface . Coronavirus in humans causes a range of disorders, from mild respiratory tract infections to lethal infections .'}


A voir

In [37]:
# https://www.machinelearningplus.com/nlp/text-summarization-approaches-nlp-example/
# https://towardsdatascience.com/abstractive-summarization-using-pytorch-f5063e67510
# https://www.analyticsvidhya.com/blog/2021/10/text-summarization-using-the-conventional-hugging-face-transformer-and-cosine-similarity/


# Sauvegarde du fichier final en csv

In [38]:
df_emails.body[32]

'[\'\', \'---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/26/2000 \', \'11:57 AM ---------------------------\', \'\', \'\', \'"BS Stone" <bs_stone@yahoo.com> on 09/26/2000 04:47:40 AM\', \'To: "jeff" <jeff@freeyellow.com>\', \'cc: "Phillip K Allen" <Phillip.K.Allen@enron.com> \', \'Subject: closing\', \'\', \'\', \'\', \'Jeff, \', \'?\', "Is the closing today?? After reviewing the  agreement?I find it isn\'t binding ", "as far as I can determine.? It is  too vague and it doesn\'t sound like ", \'anything an attorney or title company  would?draft for a real estate \', \'closing--but, of course, I could be  wrong.? \', \'?\', \'If this?closing is going to take place without  this agreement then there is \', "no point in me  following up on this?document\'s validity.? ", \'?\', "I will just need to go back to my closing documents  and see what\'s there and ", \'find out where I am with that and deal with this as  best I can.\', \'?\', \'I guess I was expecting something 

In [39]:
df_emails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   date                 100 non-null    object
 1   from                 100 non-null    object
 2   to                   100 non-null    object
 3   header               60 non-null     object
 4   body                 100 non-null    object
 5   NER_header           100 non-null    object
 6   NER_body             100 non-null    object
 7   body_dict            100 non-null    object
 8   summary_TFIDF        100 non-null    object
 9   summary_spacy        100 non-null    object
 10  summary_abstractive  100 non-null    object
 11  summary_gensim       100 non-null    object
dtypes: object(12)
memory usage: 9.5+ KB


In [40]:
# SAUVEGARDE en csv du résultat.
file_name = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique_tmp.csv"
df.to_csv(file_name, encoding='utf-8', index=False)

# Archived